In [ ]:
#| label: data-setup

# Import necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
from datetime import datetime

# Load the historical data
historic_bloom = pd.read_csv("../data/analysis_data/historic_bloom.csv")

# Load the modern data
modern_bloom = pd.read_csv("../data/analysis_data/modern_bloom.csv")

# Truncate temperature to 1 decimal point
historic_bloom['avg_temp_march_c'] = historic_bloom['avg_temp_march_c'].round(1)
modern_bloom['avg_temp_march_c'] = modern_bloom['avg_temp_march_c'].round(1)

historic = historic_bloom
modern = modern_bloom

# Drop NaN values
historic = historic.dropna()
modern = modern.dropna()

# Introduction

Before the advent of modern technology, people understood themselves, their environments, and their cultures through the cycle of life and death. The Japanese tradition of *hanami* (loosely translated as 'flower-viewing') is an important cultural tradition that honors this perspective, being associated with the flowering birth of cherry blossom trees in the late spring. In the city of Kyoto in particular, local newspapers have recorded the dates of tree flowering since the late 19th century, but researchers like @Aono2010 have been able to estimate flowering dates as far back as the year 812 CE, by combing through historical records such as diaries, poetry and other written documents describing *hanami* parties throughout the ages. However, rising global temperatures create an anomaly, highlighting the need to understand the historical and modern links between temperature and cherry blossom bloom timing. Without it, cherished traditions like *hanami* may struggle to adapt to climate shifts, risking their disappearance as a unique Japanese custom. To this end, work like @Allen2014's analyse and relatively modern blooming data over a 29-year period, but the key weakness of this is a lack of perspective. Given that their data only covers a small part of the entire recorded history of cherry blossom blooming, their analysis and subsequent model have a key weakness - they do not take into account large scale trends of blooming and their relationship to the change in climate that has happened in the last 150 years, which @Aono2010's data can illustrate.

We seek to answer the question of whether warmer temperatures lead to earlier blooming times. To do this, we analyse and model the nearly 1200-year record compiled by @Aono2010 in tandem with modern data compiled by the Japanese Meteorological Agency (JMA), collected and compiled together by @Cookson. @sec-lit-review does a broad review of existing work surrounding cherry blossoms, their significance, and efforts to understand their behaviour related to climate. @sec-data covers the data sources, methodology, and conducts an exploratory analysis of the dataset. @sec-model proposes a model of cherry blossom flowering day as a function of mean March temperature in Kyoto and lists its results. @sec-discussion critically examines the data and results, comes to several conclusions about the dataset, and proposes new areas of exploration. This analysis highlights several conclusions. First, temperature changes will have consequential effects for wildlife that function around the timing of blooming *sakura*. Next, social celebrations such as *hanami* will be impacted, changing the timing of a once-constant annual event. Finally, there is a need for more a more comprehensive inquiry into the determinants of cherry blossom blooming in Japan more generally, involving more covariates, so that more robust predictions of *sakura* blooming in Kyoto can be made. By gathering more data to make better predictions, the city of Kyoto will better be able to weather climate change and bridge their cherished traditions into the future. 

# Literature Review {#sec-lit-review}

The relationship between sakura blooming date and other factors was first examined in depth. A notable amount of scholarship addresses the relationship between different variables and and the flowering dates of sakura, from humanistic cultural studies perspectives ranging to scientific plant biology, phenological modeling, and climatology studies.

@Acevedo2021 provides a rich cultural context for understanding cherry blossom phenology in Kyoto. Her research highlights a 1200-year flowering record of *Prunus jamasakura* in Kyoto, describing it as "the longest and most complete phenological record in existence." These records, compiled from imperial court documents, poetry, and newspaper accounts of *hanami* celebrations, reveal that Kyoto's cherry trees have responded to major climate fluctuations throughout history, with progressive advancement in bloom dates occurring after approximately 1830. This shift aligns with the end of the Little Ice Age and the onset of modern warming trends. Beyond scientific significance, Acevedo explores how sakura in Kyoto have traditionally served as indicators of agricultural prosperity, with early-falling petals considered inauspicious for rice harvests, demonstrating their enduring role as both cultural and climatic storytellers in Japanese society.

According to @Aono2008, cherry trees in Kyoto are currently experiencing warmer conditions than at any time in the past twelve centuries, resulting in average flowering dates that are seven days earlier in the 1971-2000 period compared to historical averages.

Next, Xu et al.'s research provides significant physiological evidence supporting the temperature-flowering relationship in cherry trees [@Xu2023]. Their study not only documented how temperature fluctuations caused 2-3 day shifts in flowering periods between 2016-2017, but also established varying cold tolerance rankings among cherry varieties that influence bloom timing. The authors revealed the underlying hormonal mechanisms—specifically changes in ABA, ZR, IAA, and GA3 levels—that regulate flower bud differentiation in response to temperature changes. This work strengthens our understanding of how temperature directly affects cherry blossom phenology through specific physiological pathways, offering valuable context for examining historical sakura flowering records in Kyoto and predicting how future climate patterns might alter traditional bloom timing.

@Allen2014, "Modeling daily flowering probabilities" offers an approach to rigorous quantitative predictions about cherry blossom phenology under climate change in Japan. They analyzed four cherry taxa (Prunus spachiana, P. × yedoensis, P. jamasakura, and P. lannesiana) using flexible Bayesian survival regression models applied to a 29-year dataset from Tama Forest Cherry Preservation Garden in Hachioji. Their approach incorporated both chilling requirements and heat accumulation mechanisms, finding that early-flowering species like *P. spachiana* were particularly sensitive to warming temperatures. The models predicted that all studied cherry taxa will flower approximately 30 days earlier by 2100 under the A1B emissions scenario, with 2-6 days greater uncertainty around mean flowering dates. Allen et al. found that sequential models (where trees require chilling before responding to warming) better predicted flowering times than simpler approaches. The study demonstrates that while cherry trees will continue to meet their chilling requirements under projected climate scenarios, accelerated heat accumulation will drive the dramatic advancement in flowering times.

# Data {#sec-data}

To investigate the relationship between mean March temperature and blooming date in Kyoto, historical data on cherry blossom blooming dates [@Aono2010], as well as data collected through modern measurement techniques [@JMA_sakura] [@JMA_temp] were analyzed. @Cookson provided both datasets assembled in a single repository, though the modern data sources were last updated in 2020.

All of the data was transformed in Python [@python], using the `pandas` [@pandas] and `numpy` [@numpy] packages. For this analysis, the average (mean) march temperature, bloom date, day of year of blooming and year were made of interest.

In the historical data, @Aono2010 reconstructs the mean march temperature in Kyoto for some of the years, based on the blooming date, but also provided observed temperatures taken from local newspapers for a given year when possible. We prioritized used observed temperatures, but if none were available, then the reconstructed one was used. This created a dataset of 827 observations, from the year 812-2015. A sample of the table can be seen in @tbl-historical.


In [ ]:
#| label: tbl-historical-data
#| tbl-cap: Historical Cherry Blossom Blooming

# Rename columns for better readability
historic_bloom_renamed = historic_bloom.rename(columns={
    'year': 'Year',
    'flower_date': 'Bloom Date',
    'flower_doy': 'Day of Year',
    'avg_temp_march_c': 'March Temp (°C)'
})
# Display the table without .style.set_caption()
display(historic_bloom_renamed.head())

With the modern data, @Cookson combined average monthly temperatures and blooming date in a single dataset for several cities across Japan. We filtered this data to solely Kyoto, and solely average march temperatures to match the historical data from @Aono2010. This created a dataset with 67 observations ranging from 1953 to 2015. A sample can be seen in @tbl-modern.


In [ ]:
#| label: tbl-modern-data
#| tbl-cap: Modern Cherry Blossom Blooming

# Rename columns for better readability
modern_bloom_renamed = modern_bloom.rename(columns={
    'year': 'Year',
    'flower_date': 'Bloom Date',
    'flower_doy': 'Day of Year',
    'avg_temp_march_c': 'March Temp (°C)'
})

# Display the table without .style.set_caption()
display(modern_bloom_renamed.head())

In our data exploration, the `matplotlib` [@matplotlib] and `seaborn` packages were used to visualize and explore the data. The datasets were first explored chronologically, beginning with the historical data and ending with the modern data.

HISTORICAL DATA


In [ ]:
#| label: historic-temp-yr-scatter
#| echo: false
#| warning: false
#| fig-cap: WRITE SOMETHING
#| fig-align: center

# Scatterplot of historic temp vs years 
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

# Create a custom pink/light blue color palette
pink_blue_colors = ['#FF9ECD', '#87CEFA']  # Light pink and light blue
custom_cmap = LinearSegmentedColormap.from_list('pink_blue', pink_blue_colors)

# Scatter plot with pink color
plt.scatter(historic['year'], historic['avg_temp_march_c'], alpha=0.7, color='#FF9ECD', edgecolor='#E75480', s=50)

# Add moving average line in light blue
historic['moving_avg'] = historic['avg_temp_march_c'].rolling(window=10).mean()
plt.plot(historic['year'], historic['moving_avg'], color='#4BA3C3', linewidth=2.5, label='10-Year Moving Avg')

# Add reference line for mean temperature
mean_temp = historic['avg_temp_march_c'].mean()
plt.axhline(y=mean_temp, color='#87CEFA', linestyle='--', alpha=0.8, 
            label=f'Historical Mean ({mean_temp:.1f}°C)')

plt.legend(framealpha=0.7)

# Customize the plot with pink/blue theme
plt.title('Historic Temperature vs Year', fontsize=16, color='#E75480')
plt.xlabel('Year', fontsize=12, color='#4BA3C3')
plt.ylabel('Temperature (°C)', fontsize=12, color='#4BA3C3')
plt.xticks(rotation=45, color='#4F86C6')
plt.yticks(color='#4F86C6')
plt.grid(True, alpha=0.3, color='#E0E0E0')

# Add pink/blue background touches
plt.gca().set_facecolor('#F8F8FF')  # Light background
plt.gcf().set_facecolor('#F8F8FF')

plt.tight_layout()
plt.show()

This scatterplot displays historical annual temperature records (in °C) from the year 800 CE to 2000 CE. Each pink dot represents the recorded temperature for a given year, while the solid teal line represents a 10-year moving average, smoothing out year-to-year fluctuations to highlight longer-term trends. The dashed horizontal line represents the historical mean temperature of 6.2°C. Notably, the data show considerable fluctuations over the centuries, with a relatively stable pattern through much of the pre-industrial era. However, a sharp and continuous rise in average temperature is observed beginning in the late 19th to early 20th century, suggesting significant warming recently. This trend aligns with broader patterns of climate change and may have implications for phenological events such as the timing of sakura (cherry blossom) flowering.


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: WRITE SOMETHING
#| fig-align: center

# Scatterplot of historic year vs flowering day
plt.figure(figsize=(12, 7))

# Filter out NaN values
filtered_historic = historic.dropna(subset=['flower_doy'])

# Create scatter plot with pink color
plt.scatter(filtered_historic['year'], filtered_historic['flower_doy'], alpha=0.7, color='#FF9ECD', 
            edgecolor='#E75480', s=50, label='Flowering Day')

# Add moving average line with light blue color
window_size = 10  # 10-year moving average
filtered_historic['moving_avg_doy'] = filtered_historic['flower_doy'].rolling(window=window_size).mean()
plt.plot(filtered_historic['year'], filtered_historic['moving_avg_doy'], color='#4BA3C3', linewidth=2.5, 
         label=f'{window_size}-Year Moving Avg')

# Add reference line for mean with light blue
plt.axhline(y=mean_doy, color='#87CEFA', linestyle='--', alpha=0.8, 
            label=f'Historical Mean ({mean_doy:.1f}, April 13-14)')

# Customize the plot with pink/blue theme
plt.title('Cherry Blossom Flowering Day Throughout History', fontsize=16, color='#E75480')
plt.xlabel('Year', fontsize=12, color='#4BA3C3')
plt.ylabel('Day of Year (DOY)', fontsize=12, color='#4BA3C3')
plt.grid(True, alpha=0.3, color='#E0E0E0')
plt.xticks(color='#4F86C6')
plt.yticks(color='#4F86C6')
plt.legend(framealpha=0.7)

# Create a secondary y-axis with date labels
ax = plt.gca()
ax2 = ax.twinx()

# Get the current y-limits from the primary axis
y_min, y_max = ax.get_ylim()

# Create properly aligned DOY-to-date mapping
# In a non-leap year:
# DOY 90 = March 31
# DOY 91 = April 1
# DOY 121 = May 1

doy_values = np.arange(int(y_min), int(y_max) + 1, 5)  # Create ticks every 5 days
date_labels = []

for doy in doy_values:
    if doy < 90:
        # DOYs below 90 (March 31)
        day_in_mar = doy - 59  # 59 = DOYs in Jan (31) + Feb (28)
        date_labels.append(f'Mar {day_in_mar}')
    elif doy < 120:
        # DOYs for April (90-119)
        day_in_apr = doy - 90 + 1
        date_labels.append(f'Apr {day_in_apr}')
    elif doy < 151:
        # DOYs for May (120-150)
        day_in_may = doy - 120 + 1
        date_labels.append(f'May {day_in_may}')
    else:
        # DOYs for June and beyond
        day_in_jun = doy - 151 + 1
        date_labels.append(f'Jun {day_in_jun}')

# Set the properly aligned ticks and labels
ax2.set_ylim(y_min, y_max)
ax2.set_yticks(doy_values)
ax2.set_yticklabels(date_labels)
ax2.set_ylabel('Approximate Date', color='#4BA3C3', fontsize=14)
ax2.tick_params(axis='y', colors='#4F86C6')

plt.legend(framealpha=0.7)
plt.grid(True, alpha=0.3, axis='y')


# Add pink/blue background touches
plt.gca().set_facecolor('#F8F8FF')  # Light background
plt.gcf().set_facecolor('#F8F8FF')

plt.tight_layout()
plt.show()

This graph showcases the annual full flowering dates of sakura from 800 to 2000 CE, plotted as day-of-year (DOY) values, where higher values indicate later blooming in the calendar year. Each pink dot corresponds to an individual year's observed full bloom date, and the solid teal line shows a 10-year moving average; the dashed horizontal line indicates the historical mean, which is 104.6. The data suggest that, while flowering dates have fluctuated across centuries, a notable trend toward earlier blooming began in the late 19th and 20th centuries. This shift coincides with the period of modern climate warming seen in the corresponding temperature graph, supporting the hypothesis that rising spring temperatures are advancing sakura bloom time. 


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: WRITE SOMETHING
#| fig-align: center


# Violin Plot 

# Create a figure
plt.figure(figsize=(14, 8))

# Define correct century order with 9th century first
century_order = ['9th', '10th', '11th', '12th', '13th', '14th', '15th', 
                '16th', '17th', '18th', '19th', '20th', '21st']

# Create a custom color palette that transitions from light blue to pink
start_color = '#ADD8E6'  # Light blue
end_color = '#FFC0CB'    # Light pink

# Generate a custom colormap
n_colors = len(century_order)
colors = [start_color, end_color]
cmap_name = 'blue_to_pink'
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_colors)

# Get the colors for each century
custom_palette = {}
for i, century in enumerate(century_order):
    custom_palette[century] = cm(i/float(n_colors-1))

# Plot the violin plot with the custom order and palette
sns.violinplot(x='century_label', y='flower_doy', data=historic, 
               palette=custom_palette, inner='quartile', order=century_order)

# Add individual data points with small jitter
sns.stripplot(x='century_label', y='flower_doy', data=historic, 
              color='#FF9ECD', alpha=0.3, jitter=True, size=4, order=century_order)

# Add reference line for mean flowering day (assuming mean_doy is defined elsewhere)
mean_doy = historic['flower_doy'].mean()  # Calculate if not already defined
plt.axhline(y=mean_doy, color='#E75480', linestyle='--', alpha=0.8, 
            label=f'Historical Mean ({mean_doy:.1f}, April 14)')

# Enhance visuals
plt.title('Cherry Blossom Flowering Day Distribution by Century', fontsize=18, color='#E75480')
plt.xlabel('Century', fontsize=14, color='#4BA3C3')
plt.ylabel('Day of Year (DOY)', fontsize=14, color='#4BA3C3')
plt.xticks(rotation=45, color='#4F86C6')
plt.yticks(color='#4F86C6')

# Create a secondary y-axis with date labels
ax = plt.gca()
ax2 = ax.twinx()

# Get the current y-limits from the primary axis
y_min, y_max = ax.get_ylim()

# Create properly aligned DOY-to-date mapping
# In a non-leap year:
# DOY 90 = March 31
# DOY 91 = April 1
# DOY 121 = May 1

doy_values = np.arange(int(y_min), int(y_max) + 1, 5)  # Create ticks every 5 days
date_labels = []

for doy in doy_values:
    if doy < 90:
        # DOYs below 90 (March 31)
        day_in_mar = doy - 59  # 59 = DOYs in Jan (31) + Feb (28)
        date_labels.append(f'Mar {day_in_mar}')
    elif doy < 120:
        # DOYs for April (90-119)
        day_in_apr = doy - 90 + 1
        date_labels.append(f'Apr {day_in_apr}')
    elif doy < 151:
        # DOYs for May (120-150)
        day_in_may = doy - 120 + 1
        date_labels.append(f'May {day_in_may}')
    else:
        # DOYs for June and beyond
        day_in_jun = doy - 151 + 1
        date_labels.append(f'Jun {day_in_jun}')

# Set the properly aligned ticks and labels
ax2.set_ylim(y_min, y_max)
ax2.set_yticks(doy_values)
ax2.set_yticklabels(date_labels)
ax2.set_ylabel('Approximate Date', color='#4BA3C3', fontsize=14)
ax2.tick_params(axis='y', colors='#4F86C6')

plt.legend(framealpha=0.7)
plt.grid(True, alpha=0.3, axis='y')

# Set background color
plt.gca().set_facecolor('#F8F8FF')
plt.gcf().set_facecolor('#F8F8FF')

plt.tight_layout()
plt.show()

The violin plot above showcases the cherry blossom flowering date distribution by century (9th to 21st), measured in day-of-year (DOY). The dashed pink line represents the average flowering day of the year (~105 days). It was quite consistent until the 20th century (1900s), where we can see a large drop; this trend has also been carried into the 21st century, where it even dropped lower. 

MODERN DATA


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: WRITE SOMETHING
#| fig-align: center

# Scatterplot of modern year vs flowering day
plt.figure(figsize=(12, 7))

# Filter out NaN values
filtered_modern = modern.dropna(subset=['flower_doy'])

# Create scatter plot with pink color
plt.scatter(filtered_modern['year'], filtered_modern['flower_doy'], alpha=0.7, color='#FF9ECD', 
            edgecolor='#E75480', s=50, label='Flowering Day')

# Add moving average line with light blue color
window_size = 5  # 5-year moving average
filtered_modern['moving_avg_doy'] = filtered_modern['flower_doy'].rolling(window=window_size).mean()
plt.plot(filtered_modern['year'], filtered_modern['moving_avg_doy'], color='#4BA3C3', linewidth=2.5, 
         label=f'{window_size}-Year Moving Avg')

# Customize the plot with pink/blue theme
plt.title('Cherry Blossom Flowering Day Throughout Modern Times', fontsize=16, color='#E75480')
plt.xlabel('Year', fontsize=12, color='#4BA3C3')
plt.ylabel('Day of Year (DOY)', fontsize=12, color='#4BA3C3')
plt.grid(True, alpha=0.3, color='#E0E0E0')
plt.xticks(color='#4F86C6')
plt.yticks(color='#4F86C6')
plt.legend(framealpha=0.7)

# Create a secondary y-axis with date labels
ax = plt.gca()
ax2 = ax.twinx()

# Get the current y-limits from the primary axis
y_min, y_max = ax.get_ylim()

# Create properly aligned DOY-to-date mapping
# In a non-leap year:
# DOY 90 = March 31
# DOY 91 = April 1
# DOY 121 = May 1

doy_values = np.arange(int(y_min), int(y_max) + 1, 5)  # Create ticks every 5 days
date_labels = []

for doy in doy_values:
    if doy < 90:
        # DOYs below 90 (March 31)
        day_in_mar = doy - 59  # 59 = DOYs in Jan (31) + Feb (28)
        date_labels.append(f'Mar {day_in_mar}')
    elif doy < 120:
        # DOYs for April (90-119)
        day_in_apr = doy - 90 + 1
        date_labels.append(f'Apr {day_in_apr}')
    elif doy < 151:
        # DOYs for May (120-150)
        day_in_may = doy - 120 + 1
        date_labels.append(f'May {day_in_may}')
    else:
        # DOYs for June and beyond
        day_in_jun = doy - 151 + 1
        date_labels.append(f'Jun {day_in_jun}')

# Set the properly aligned ticks and labels
ax2.set_ylim(y_min, y_max)
ax2.set_yticks(doy_values)
ax2.set_yticklabels(date_labels)
ax2.set_ylabel('Approximate Date', color='#4BA3C3', fontsize=14)
ax2.tick_params(axis='y', colors='#4F86C6')

plt.legend(framealpha=0.7)
plt.grid(True, alpha=0.3, axis='y')


# Add pink/blue background touches
plt.gca().set_facecolor('#F8F8FF')  # Light background
plt.gcf().set_facecolor('#F8F8FF')

plt.tight_layout()
plt.show()

The scatterplot above presents the annual full flowering dates of cherry blossoms in Japan from 1950 to 2020, measured in day-of-year (DOY), where higher values represent later blooming. Each pink dot marks the full flowering day for a given year, while the solid blue line shows a 5-year moving average, smoothing short-term fluctuations to reveal broader temporal trends. Over the 70-year span, there is a clear downward trend, indicating progressively earlier flowering.


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: WRITE SOMETHING
#| fig-align: center

# scatterplot of modern data - year vs. Temperature

# Scatterplot of modern temp vs years 
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

# Create a custom pink/light blue color palette
pink_blue_colors = ['#FF9ECD', '#87CEFA']  # Light pink and light blue
custom_cmap = LinearSegmentedColormap.from_list('pink_blue', pink_blue_colors)

# Scatter plot with pink color
plt.scatter(modern['year'], modern['avg_temp_march_c'], alpha=0.7, color='#FF9ECD', edgecolor='#E75480', s=50)

# Add moving average line in light blue
modern['moving_avg'] = modern['avg_temp_march_c'].rolling(window=5).mean()  # Using 5-year window for modern data
plt.plot(modern['year'], modern['moving_avg'], color='#4BA3C3', linewidth=2.5, label='5-Year Moving Avg')

plt.legend(framealpha=0.7)

# Customize the plot with pink/blue theme
plt.title('Modern Temperature vs Year', fontsize=16, color='#E75480')
plt.xlabel('Year', fontsize=12, color='#4BA3C3')
plt.ylabel('Temperature (°C)', fontsize=12, color='#4BA3C3')
plt.xticks(rotation=45, color='#4F86C6')
plt.yticks(color='#4F86C6')
plt.grid(True, alpha=0.3, color='#E0E0E0')

# Add pink/blue background touches
plt.gca().set_facecolor('#F8F8FF')  # Light background
plt.gcf().set_facecolor('#F8F8FF')

plt.tight_layout()
plt.show()

This scatterplot shows the annual spring temperature in Japan from 1950 to 2020, measured in degrees Celsius (°C) relative to a baseline average. Each pink dot represents the average March temperature for a given year, and the solid blue line indicates a 5-year moving average to highlight underlying climate trends. The overall trajectory shows a clear increase in average March temperatures over the 70-year period, particularly after 1990, reflecting the impact of modern global climate change. This warming trend corresponds closely with the earlier flowering of cherry blossoms during the same time frame shown in the earlier figure. Together, both graphs reinforce the connection between rising spring temperatures and timing shifts in sakura blooming.


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: WRITE SOMETHING
#| fig-align: center

# histogram of flowering days
plt.figure(figsize=(12, 7))
plt.hist(modern['flower_doy'], bins=30, color='#FF9ECD', edgecolor='#E75480', alpha=0.7)
plt.title('Distribution of Flowering Days (Modern)', fontsize=18, color='#E75480')
plt.xlabel('Day of Year (DOY)', fontsize=14, color='#4BA3C3')
plt.ylabel('Frequency', fontsize=14, color='#4BA3C3')
plt.xticks(color='#4F86C6')
plt.yticks(color='#4F86C6')
plt.legend(framealpha=0.7)
plt.grid(True, alpha=0.3, color='#E0E0E0')
plt.gca().set_facecolor('#F8F8FF')  # Light background
plt.gcf().set_facecolor('#F8F8FF')
plt.tight_layout()
plt.show()

This histogram highlights the frequency distribution of sakura full flowering dates in modern Japan, measured in day-of-year (DOY), for the period between 1950 and 2020. Each bar represents how often flowering occurred within a specific DOY range. The distribution is roughly bell-shaped and unimodal, with the majority of flowering dates clustered between DOY 83 and DOY 95 (approximately March 24 to April 4), suggesting a central tendency in late March. 


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: WRITE SOMETHING
#| fig-align: center

# histogram of temperatures
plt.figure(figsize=(12, 7))
plt.hist(modern['avg_temp_march_c'], bins=30, color='#FF9ECD', edgecolor='#E75480', alpha=0.7)
plt.title('Distribution of Average March Temperatures (Modern)', fontsize=18, color='#E75480')
plt.xlabel('Temperature (°C)', fontsize=14, color='#4BA3C3')
plt.ylabel('Frequency', fontsize=14, color='#4BA3C3')
plt.xticks(color='#4F86C6')
plt.yticks(color='#4F86C6')
plt.legend(framealpha=0.7)
plt.grid(True, alpha=0.3, color='#E0E0E0')
plt.gca().set_facecolor('#F8F8FF')  # Light background
plt.gcf().set_facecolor('#F8F8FF')
plt.tight_layout()
plt.show()

The figure above presents the frequency distribution of average March temperatures in Japan from 1950 to 2020. The temperature values are plotted on the x-axis in degrees Celsius (°C). The distribution is centered around -1.5°C, with a wider spread toward colder extremes and a thinner tail extending into positive temperatures. While most March temperatures during this period fall below freezing, the presence of warmer years, particularly in the latter decades, indicates a shift in climate norms. These warming trends align with observed shifts in cherry blossom phenology, where earlier flowering is increasingly correlated with milder spring conditions.

## Data Exploration

# Modeling {#sec-model}

asl;dkfj;laksdfjl;asjdf;lasjdfljasl;fj;lasjdf

## Model Specification

## Results

# Discussion {#sec-discussion}

alsdhj;lasjfljas;lfja;lsjf;lasjdf;

## Ecological Implications

## Cultural Implications

## Limitations

## Future Research Directions

# References

# Appendix {#sec-appendix}